## Causal estimation

This notebook provides an attempt at causal estimation of the role of gender in wage inequalities. 
As a reminder, this dataset  - `Description des emplois salariés en 2021` is taken from the `Insee` website at the following link : <https://www.insee.fr/fr/statistiques/7651654#dictionnaire>.
We aim to study the effect of gender on the level of wages, depending on several variables.

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon
from tqdm import tqdm
import gdown
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode
init_notebook_mode(connected= True)
import plotly.express as px
import seaborn as sns
import scipy as sp

`Warning` : This code should be run after the [import and formating notebook](test_import_données.ipynb).

In [ ]:
base = pd.read_csv("INSEE_DATA_TREATED.csv")

In [ ]:
base.columns

In [ ]:
target = ['WAGE']
numerical_columns = [
    #'DATDEB', 'DATFIN', #date début et fin de rémunération par rapport au 01/01
     'AGE', #age en années
    'DUREE', #durée de paie en jours
      'NBHEUR', 'NBHEUR_TOT', #nombre d'heures salariées total (quelle diff?)
    #'WAGE', #transformation of TRNNETO
    #'UNEMP' #transformation of TRALCHT
]

categorical_columns = [
    #'A6', 'A17', 'A38' #activité en nomenclature agrégrée
    'CPFD', #temps complet ou partiel
    #DEPR', 'DEPT', #département résidence et travail
    #'DOMEMPL', 'DOMEMPL_EM', #domaine de l'emploi et l'établissement d'affectation/employeur
    
    'FILT', #indic poste annexe 2 ou non-annexe 1 (seuils rémunération volume)
    #'REGR', 'REGT', #région de résidence et de travail
    'SEXE', #1 homme 2 femme
    'PCS', #PCS-ESE
    'TYP_EMPLOI', #ordinaire, apprenti, autre
    #'CONV_COLL', #convention collective
  
    #'TRNNETO', #rémunération nette globale en tranches -> à passer en numérique ?
    'TRALCHT', #total des indémnités de chômage, en tranches -> passage en numérique ?
    'TREFF', #tranche d'effectif : de 0 à 250+ postes
    'CONT_TRAV', #contrat de travail : APP apprentissage, TOA occasionnel ou à l'acte, TTP intérim, AUTre
    'CS', #CSP mais code plus simple 
    'AGE_TR', #age en tranches quadriennales
    'DATDEB_TR',
       'DATFIN_TR', #dates début et fin rémunération en tranches
    #'DUREE_TR', #durée de paie exprimée en jours en tranches mensuelles
    'DOMEMPL_EM_N', 'DOMEMPL_N', 'REGR_N',
       'REGT_N', 'CS_N', 'DEPR_N', 'DEPT_N','A38_N' #les variables renommées avec les labels correspondants aux codes
]

all_columns = numerical_columns + categorical_columns

## First method: a penalized linear model 

This step is useful to get a first broad idea of which covariates are important to predict wages.

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import PredictionErrorDisplay, median_absolute_error

First, we split the sample into a train and a test dataset. 
Only the train dataset will be used to fit the model.

In [ ]:
X = base[all_columns]
X.describe(include="all")

We modify the target variable (wage) by replacing the 0s with 1s that can be handled when transformed by a log, this allows us to avoid infinity values.

In [ ]:
base['TARGET'] = base[target]
base.loc[base['TARGET'] == 0, 'TARGET'] = 1
y = base['TARGET']
y.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

The datasat contains categorical variables that cannot be included in a linear model if they are not coded as integers first. In addition, to avoid categorical features to be treated as ordered values, we need to one-hot-encode them.

We will do that with a pre-processor that:
- one-hot-encode the categorical columns
- rescale numerical columns.

In [ ]:
#preprocessor = make_column_transformer(
 #   (OneHotEncoder(drop="if_binary"), categorical_columns),
 #   (StandardScaler(), numerical_columns),
#)

We modify the preprocessing step to handle missing values.
We could have also dropped all rows with missing information.

In [ ]:
preprocessor = make_column_transformer(
    # For categorical columns: impute missing values with the most frequent value and apply OneHotEncoding
    (make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(drop="if_binary")), categorical_columns),
    
    # For numerical columns: impute missing values with the median and apply StandardScaler
    (make_pipeline(SimpleImputer(strategy='median'), StandardScaler()), numerical_columns)
)

In [ ]:
model = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=Ridge(alpha=1e-10), func=np.log10, inverse_func=sp.special.exp10
    ),
)

In [ ]:
model.fit(X_train, y_train)

We check the performance of the model by checking its predictions on the test set with the median absolute error.

In [ ]:
mae_train = median_absolute_error(y_train, model.predict(X_train))
y_pred = model.predict(X_test)
mae_test = median_absolute_error(y_test, y_pred)
scores = {
    "MedAE on training set": f"{mae_train:.2f} €/year",
    "MedAE on testing set": f"{mae_test:.2f} €/year",
}

In [ ]:
_, ax = plt.subplots(figsize=(5, 5))
display = PredictionErrorDisplay.from_predictions(
    y_test, y_pred, kind="actual_vs_predicted", ax=ax, scatter_kwargs={"alpha": 0.5}
)
ax.set_title("Ridge model, small regularization")
for name, score in scores.items():
    ax.plot([], [], " ", label=f"{name}: {score}")
ax.legend(loc="upper left")
plt.tight_layout()

In [ ]:
feature_names = model[:-1].get_feature_names_out()
coefs = pd.DataFrame(
    model[-1].regressor_.coef_,
    columns=["Coefficients importance"],
    index=feature_names,
)
coefs.plot.barh(figsize=(9, 7))
plt.title("Ridge model, small regularization, normalized variables")
plt.xlabel("Raw coefficient values")
plt.axvline(x=0, color=".5")
plt.subplots_adjust(left=0.3)

In [ ]:
coefs_005=coefs[coefs["Coefficients importance"]>0.05]

In [ ]:
coefs_005.plot.barh(figsize=(9, 7))
plt.title("Ridge model, small regularization, normalized variables")
plt.xlabel("Raw coefficient values")
plt.axvline(x=0, color=".5")
plt.subplots_adjust(left=0.3)

In [ ]:
coefs_minus005=coefs[coefs["Coefficients importance"]<-0.05]

coefs_minus005.plot.barh(figsize=(9, 7))
plt.title("Ridge model, small regularization, normalized variables")
plt.xlabel("Raw coefficient values")
plt.axvline(x=0, color=".5")
plt.subplots_adjust(left=0.3)

In [ ]:
coefs

In [ ]:
plt.hist(model[1].regressor_.coef_,bins=model[1].regressor_.coef_.shape[0])

## First model : less variables

In [ ]:
target = ['WAGE']
numerical_columns = [
    #'DATDEB', 'DATFIN', #date début et fin de rémunération par rapport au 01/01
     'AGE', #age en années
    'DUREE', #durée de paie en jours
      #'NBHEUR', 
    'NBHEUR_TOT', #nombre d'heures salariées total (quelle diff?)
    #'WAGE', #transformation of TRNNETO
    #'UNEMP' #transformation of TRALCHT
]

categorical_columns = [
    #'A6', 'A17', 'A38' #activité en nomenclature agrégrée
    'CPFD', #temps complet ou partiel
    #DEPR', 'DEPT', #département résidence et travail
    #'DOMEMPL', 'DOMEMPL_EM', #domaine de l'emploi et l'établissement d'affectation/employeur
    
    'FILT', #indic poste annexe 2 ou non-annexe 1 (seuils rémunération volume)
    #'REGR', 'REGT', #région de résidence et de travail
    'SEXE', #1 homme 2 femme
    #'PCS', #PCS-ESE
    'TYP_EMPLOI', #ordinaire, apprenti, autre
    #'CONV_COLL', #convention collective
  
    #'TRNNETO', #rémunération nette globale en tranches -> à passer en numérique ?
    'TRALCHT', #total des indémnités de chômage, en tranches -> passage en numérique ?
    'TREFF', #tranche d'effectif : de 0 à 250+ postes
    'CONT_TRAV', #contrat de travail : APP apprentissage, TOA occasionnel ou à l'acte, TTP intérim, AUTre
    'CS', #CSP mais code plus simple 
    'AGE_TR', #age en tranches quadriennales
    'DATDEB_TR',
       'DATFIN_TR', #dates début et fin rémunération en tranches
    #'DUREE_TR', #durée de paie exprimée en jours en tranches mensuelles
    'DOMEMPL_EM_N', 'DOMEMPL_N', 'REGR_N',
       'REGT_N', 'CS_N', 
    #'DEPR_N', 'DEPT_N','A38_N' #les variables renommées avec les labels correspondants aux codes
]

all_columns = numerical_columns + categorical_columns

In [ ]:
X = base[all_columns]
X.describe(include="all")
base['TARGET'] = base[target]
base.loc[base['TARGET'] == 0, 'TARGET'] = 1
y = base['TARGET']
y.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

We use the same model :

In [ ]:
preprocessor = make_column_transformer(
    # For categorical columns: impute missing values with the most frequent value and apply OneHotEncoding
    (make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(drop="if_binary")), categorical_columns),
    
    # For numerical columns: impute missing values with the median and apply StandardScaler
    (make_pipeline(SimpleImputer(strategy='median'), StandardScaler()), numerical_columns)
)

In [ ]:
model = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=Ridge(alpha=1e-10), func=np.log10, inverse_func=sp.special.exp10
    ),
)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
feature_names = model[:-1].get_feature_names_out()
coefs = pd.DataFrame(
    model[-1].regressor_.coef_,
    columns=["Coefficients importance"],
    index=feature_names,
)
coefs.plot.barh(figsize=(9, 7))
plt.title("Ridge model, small regularization, normalized variables")
plt.xlabel("Raw coefficient values")
plt.axvline(x=0, color=".5")
plt.subplots_adjust(left=0.3)

## Second model: Lasso

In [ ]:
from sklearn.linear_model import LassoCV


In [ ]:
target = ['WAGE']
numerical_columns = [
    'DATDEB', 'DATFIN', #date début et fin de rémunération par rapport au 01/01
     'AGE', #age en années
    'DUREE', #durée de paie en jours
      'NBHEUR', 'NBHEUR_TOT', #nombre d'heures salariées total (quelle diff?)
    #'WAGE', #transformation of TRNNETO
    #'UNEMP' #transformation of TRALCHT
]

categorical_columns = [
    #'A6', 'A17', 'A38' #activité en nomenclature agrégrée
    'CPFD', #temps complet ou partiel
    #DEPR', 'DEPT', #département résidence et travail
    #'DOMEMPL', 'DOMEMPL_EM', #domaine de l'emploi et l'établissement d'affectation/employeur
    
    'FILT', #indic poste annexe 2 ou non-annexe 1 (seuils rémunération volume)
    #'REGR', 'REGT', #région de résidence et de travail
    'SEXE', #1 homme 2 femme
    'PCS', #PCS-ESE
    'TYP_EMPLOI', #ordinaire, apprenti, autre
    #'CONV_COLL', #convention collective
  
    #'TRNNETO', #rémunération nette globale en tranches -> à passer en numérique ?
    'TRALCHT', #total des indémnités de chômage, en tranches -> passage en numérique ?
    'TREFF', #tranche d'effectif : de 0 à 250+ postes
    'CONT_TRAV', #contrat de travail : APP apprentissage, TOA occasionnel ou à l'acte, TTP intérim, AUTre
    #'CS', #CSP mais code plus simple 
    'AGE_TR', #age en tranches quadriennales
    #'DATDEB_TR','DATFIN_TR', #dates début et fin rémunération en tranches
    #'DUREE_TR', #durée de paie exprimée en jours en tranches mensuelles
    'DOMEMPL_EM_N', 'DOMEMPL_N', 'REGR_N',
       'REGT_N', 'CS_N', 'DEPR_N', 'DEPT_N','A38_N' #les variables renommées avec les labels correspondants aux codes
]

all_columns = numerical_columns + categorical_columns

In [ ]:
X = base[all_columns]
X.describe(include="all")
base['TARGET'] = base[target]
base.loc[base['TARGET'] == 0, 'TARGET'] = 1
y = base['TARGET']
y.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
preprocessor = make_column_transformer(
    # For categorical columns: impute missing values with the most frequent value and apply OneHotEncoding
    (make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(drop="if_binary")), categorical_columns),
    
    # For numerical columns: impute missing values with the median and apply StandardScaler
    (make_pipeline(SimpleImputer(strategy='median'), StandardScaler()), numerical_columns)
)

We fit a Lasso model with cross-validation to choose the alpha.

# DOES NOT WORK: TOO LONG

In [ ]:

alphas = np.logspace(-10, 10, 21)  # alpha values to be chosen from by cross-validation

model = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=LassoCV(alphas=alphas, max_iter=100),
        func=np.log10,
        inverse_func=sp.special.exp10,
    ),
)

_ = model.fit(X_train, y_train)

In [ ]:
# First we verify which value of :math:`\alpha` has been selected.

model[-1].regressor_.alpha_